In [1]:
#importing libraries
import pandas as pd
import numpy as np
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
!pip install datasets
from datasets import Dataset, DatasetDict
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from transformers import EvalPrediction
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import numpy as np
from sklearn.metrics import classification_report
from collections import Counter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
file_path = "Merged/concatenated_data.txt"
df = pd.read_csv(file_path, sep="\t", header=None, names=["Document_ID", "High_Level_Narratives", "Sub_Narratives"])

In [3]:
# Group by Document_ID and create lists of narratives
df = df.groupby("Document_ID").agg({
    "High_Level_Narratives": lambda x: list(set(";".join(x).split(";"))),
    "Sub_Narratives": lambda x: list(set(";".join(x).split(";")))
}).reset_index()

df.rename(columns={
    "High_Level_Narratives": "High_Level_Narratives_List",
    "Sub_Narratives": "Sub_Narratives_List"
}, inplace=True)


In [4]:
df

,Document_ID,High_Level_Narratives_List,Sub_Narratives_List
0,EN_CC_100000.txt,"[CC: Controversy about green technologies, CC:...",[CC: Hidden plots by secret schemes of powerfu...
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Hidden plots by secret schemes of powerfu...
2,EN_CC_100003.txt,"[CC: Criticism of climate movement, CC: Critic...",[CC: Criticism of climate movement: Ad hominem...
3,EN_CC_100004.txt,"[CC: Criticism of climate movement, CC: Contro...",[CC: Criticism of climate movement: Ad hominem...
4,EN_CC_100005.txt,[CC: Criticism of climate movement],"[CC: Criticism of climate movement: Other, CC:..."
...,...,...,...
435,EN_UA_DEV_216.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...
436,EN_UA_DEV_22.txt,[URW: Blaming the war on others rather than th...,"[URW: Discrediting the West, Diplomacy: Other,..."
437,EN_UA_DEV_23.txt,"[URW: Distrust towards Media, URW: Amplifying ...",[URW: Distrust towards Media: Western media is...
438,EN_UA_DEV_24.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...


In [5]:


file_path = "Merged/subtask-2-documents"

def load_documents(folder_path):
    documents = {}
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # Ensure only text files are read
            doc_id = filename  # Extract Document_ID from filename
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                documents[doc_id] = file.read().strip()
    return documents

# Load document texts into a dictionary
document_texts = load_documents(file_path)

# Map document texts to the DataFrame
df['Text'] = df['Document_ID'].map(document_texts)

# Check if all documents are successfully mapped
print(f"Number of missing documents: {df['Text'].isnull().sum()}")
df


Mounted at /content/drive
Number of missing documents: 0


,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text
0,EN_CC_100000.txt,"[CC: Controversy about green technologies, CC:...",[CC: Hidden plots by secret schemes of powerfu...,Pentagon plans to serve LAB-GROWN MEAT to troo...
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Hidden plots by secret schemes of powerfu...,Oxford Residents Mount Resistance Against the ...
2,EN_CC_100003.txt,"[CC: Criticism of climate movement, CC: Critic...",[CC: Criticism of climate movement: Ad hominem...,"Fonda Heads To Canada For Oil Sands Protest, M..."
3,EN_CC_100004.txt,"[CC: Criticism of climate movement, CC: Contro...",[CC: Criticism of climate movement: Ad hominem...,A Tesla Owner Just Exposed A Sick Secret About...
4,EN_CC_100005.txt,[CC: Criticism of climate movement],"[CC: Criticism of climate movement: Other, CC:...",Climate Crazies Fail in Attempt to Vandalize A...
...,...,...,...,...
435,EN_UA_DEV_216.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,"EU 'biggest loser' in Ukraine conflicts, Hunga..."
436,EN_UA_DEV_22.txt,[URW: Blaming the war on others rather than th...,"[URW: Discrediting the West, Diplomacy: Other,...",European Parliament members clash over support...
437,EN_UA_DEV_23.txt,"[URW: Distrust towards Media, URW: Amplifying ...",[URW: Distrust towards Media: Western media is...,What is the current trajectory of the evil emp...
438,EN_UA_DEV_24.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,Europe ‘Shot Itself in the Lungs’ With Sanctio...


In [6]:
import pandas as pd


# Assign labels based on Document_ID
def assign_label(row):
    if row['Document_ID'].startswith("EN_CC"):
        return 1 if "CC:" in str(row['High_Level_Narratives_List']) else 0  # CC or Other
    elif row['Document_ID'].startswith("EN_UA") or row['Document_ID'].startswith("EN_UA_DEV"):
        return 2 if "URW:" in str(row['High_Level_Narratives_List']) else 0  # URW or Other
    else:
        return 0  # Default to Other

# Add the label column
df['label'] = df.apply(assign_label, axis=1)

print(df)


           Document_ID                         High_Level_Narratives_List  \
0     EN_CC_100000.txt  [CC: Controversy about green technologies, CC:...   
1     EN_CC_100002.txt  [CC: Hidden plots by secret schemes of powerfu...   
2     EN_CC_100003.txt  [CC: Criticism of climate movement, CC: Critic...   
3     EN_CC_100004.txt  [CC: Criticism of climate movement, CC: Contro...   
4     EN_CC_100005.txt                [CC: Criticism of climate movement]   
..                 ...                                                ...   
435  EN_UA_DEV_216.txt  [URW: Discrediting the West, Diplomacy, URW: N...   
436   EN_UA_DEV_22.txt  [URW: Blaming the war on others rather than th...   
437   EN_UA_DEV_23.txt  [URW: Distrust towards Media, URW: Amplifying ...   
438   EN_UA_DEV_24.txt  [URW: Discrediting the West, Diplomacy, URW: N...   
439   EN_UA_DEV_26.txt                        [URW: Russia is the Victim]   

                                   Sub_Narratives_List  \
0    [CC: Hidden 

In [7]:
df

,Document_ID,High_Level_Narratives_List,Sub_Narratives_List,Text,label
0,EN_CC_100000.txt,"[CC: Controversy about green technologies, CC:...",[CC: Hidden plots by secret schemes of powerfu...,Pentagon plans to serve LAB-GROWN MEAT to troo...,1
1,EN_CC_100002.txt,[CC: Hidden plots by secret schemes of powerfu...,[CC: Hidden plots by secret schemes of powerfu...,Oxford Residents Mount Resistance Against the ...,1
2,EN_CC_100003.txt,"[CC: Criticism of climate movement, CC: Critic...",[CC: Criticism of climate movement: Ad hominem...,"Fonda Heads To Canada For Oil Sands Protest, M...",1
3,EN_CC_100004.txt,"[CC: Criticism of climate movement, CC: Contro...",[CC: Criticism of climate movement: Ad hominem...,A Tesla Owner Just Exposed A Sick Secret About...,1
4,EN_CC_100005.txt,[CC: Criticism of climate movement],"[CC: Criticism of climate movement: Other, CC:...",Climate Crazies Fail in Attempt to Vandalize A...,1
...,...,...,...,...,...
435,EN_UA_DEV_216.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,"EU 'biggest loser' in Ukraine conflicts, Hunga...",2
436,EN_UA_DEV_22.txt,[URW: Blaming the war on others rather than th...,"[URW: Discrediting the West, Diplomacy: Other,...",European Parliament members clash over support...,2
437,EN_UA_DEV_23.txt,"[URW: Distrust towards Media, URW: Amplifying ...",[URW: Distrust towards Media: Western media is...,What is the current trajectory of the evil emp...,2
438,EN_UA_DEV_24.txt,"[URW: Discrediting the West, Diplomacy, URW: N...",[URW: Negative Consequences for the West: Sanc...,Europe ‘Shot Itself in the Lungs’ With Sanctio...,2


In [8]:
def handle_rare_classes(data, labels, min_samples=15):
    label_sums = np.sum(labels, axis=0)
    rare_classes = np.where(label_sums < min_samples)[0]

    for rare_class in rare_classes:
        rare_indices = [i for i, lbl in enumerate(labels) if lbl[rare_class] == 1]
        if len(rare_indices) > 0:
            duplicate_data = data.iloc[rare_indices]
            data = pd.concat([data] + [duplicate_data] * (min_samples - len(rare_indices)), ignore_index=True)
            labels = np.vstack([labels] + [labels[rare_indices]] * (min_samples - len(rare_indices)))
    return data, labels


In [9]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'], df['label'], test_size=0.2, random_state=42
)


In [10]:
train_texts

,Text
266,Oppose imperialist War! SEP/IYSSE public meeti...
294,UK Intelligence Experts Concerned Russia Will ...
31,Pakistan to follow China's example to enhance ...
84,Climate cultists push bizarre scare language a...
301,Russian celebrities labelled as 'foreign agent...
...,...
106,Florida Governor Follows West Virginia in Pull...
270,Russia Says US Is Responsible for Deadly Ukrai...
348,Ukraine war may bring windfall to Indonesia's ...
435,"EU 'biggest loser' in Ukraine conflicts, Hunga..."


In [11]:
train_texts,train_labels = handle_rare_classes(train_texts,train_labels)

<ipython-input-8-0d93540dbaa8>:3: DeprecationWarning: Calling nonzero on 0d arrays is deprecated, as it behaves surprisingly. Use `atleast_1d(cond).nonzero()` if the old behavior was intended. If the context of this warning is of the form `arr[nonzero(cond)]`, just use `arr[cond]`.
  rare_classes = np.where(label_sums < min_samples)[0]


In [12]:
from datasets import Dataset

# Convert train and test data to Hugging Face Datasets
train_dataset = Dataset.from_pandas(pd.DataFrame({
    "text": train_texts,
    "labels": train_labels,
    "Document_ID": df.loc[train_texts.index, "Document_ID"]
}))
test_dataset = Dataset.from_pandas(pd.DataFrame({
    "text": test_texts,
    "labels": test_labels,
    "Document_ID": df.loc[test_texts.index, "Document_ID"]
}))


# Create DatasetDict
datasets = DatasetDict({"train": train_dataset, "test": test_dataset})


In [13]:
train_dataset

Dataset({
    features: ['text', 'labels', 'Document_ID', '__index_level_0__'],
    num_rows: 352
})

In [14]:
test_dataset

Dataset({
    features: ['text', 'labels', 'Document_ID', '__index_level_0__'],
    num_rows: 88
})

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=1)

    # Adjust predictions based on document ID rules
    adjusted_preds = []
    for i, pred in enumerate(predictions):
        doc_id = tokenized_datasets["test"]["Document_ID"][i]  # Access metadata
        if doc_id.startswith("EN_CC") and pred == 2:  # Invalid URW for EN_CC
            adjusted_preds.append(1)  # Switch to CC
        elif (doc_id.startswith("EN_UA") or doc_id.startswith("EN_UA_DEV")) and pred == 1:  # Invalid CC for EN_UA
            adjusted_preds.append(2)  # Switch to URW
        else:
            adjusted_preds.append(pred)

    # Compute metrics
    precision = precision_score(labels, adjusted_preds, average="weighted")
    recall = recall_score(labels, adjusted_preds, average="weighted")
    f1 = f1_score(labels, adjusted_preds, average="weighted")
    return {"precision": precision, "recall": recall, "f1": f1}


In [16]:
tokenizer = AutoTokenizer.from_pretrained("roberta-base")
base_model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# labels = df['label'].values
# class_weights = calculate_class_weights(labels)

In [18]:
# weighted_model = WeightedModel(base_model, class_weights)

In [19]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = datasets.map(tokenize_function, batched=True)

Map:   0%|          | 0/352 [00:00<?, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [20]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [21]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=100,
    save_total_limit=1,
    seed=42,
    weight_decay=0.01,
    #fp16=True
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [22]:
trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

<ipython-input-22-dfd8459f4ee5>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [23]:
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,No log,0.540696,0.764124,0.761364,0.746552
2,0.804100,0.661766,0.778966,0.704545,0.708223
3,0.452800,0.807163,0.797448,0.784091,0.768413
4,0.431000,0.958881,0.872756,0.829545,0.814900
5,0.196800,0.724048,0.877696,0.863636,0.866281
6,0.111500,0.687717,0.889543,0.875000,0.870286
7,0.093700,1.344814,0.845960,0.806818,0.809958
8,0.009800,0.920722,0.879073,0.875000,0.875951
9,0.009800,0.745260,0.887458,0.886364,0.886728
10,0.000500,0.784384,0.899082,0.897727,0.898229


TrainOutput(global_step=1760, training_loss=0.11945489782056856, metrics={'train_runtime': 1080.3058, 'train_samples_per_second': 6.517, 'train_steps_per_second': 1.629, 'total_flos': 1852318460805120.0, 'train_loss': 0.11945489782056856, 'epoch': 20.0})

In [24]:
metrics = trainer.evaluate()
print(metrics)


{'eval_loss': 0.7843844890594482, 'eval_precision': 0.8990819654882155, 'eval_recall': 0.8977272727272727, 'eval_f1': 0.898229043683589, 'eval_runtime': 2.774, 'eval_samples_per_second': 31.723, 'eval_steps_per_second': 7.931, 'epoch': 20.0}
